## RAG

Autores :   
- Pablo Gonzalez  
- Araceli Sanchez
  
Fecha : 4/10/2025

In [7]:
%pip install langchain unstructured 

  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached python_magic-0.4.27-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached python_iso639-2025.2.18-py3-none-any.whl.metadata (14 kB)
  Using cached langdetect-1.0.9-py3-none-any.whl
  Using cached python_oxmsg-0.0.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached html5lib-1.1-py2.py3-none-any.whl.metadata (16 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
  Using cached olefile-0.47-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 16.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/608.4 kB ? eta -:--:--
   --------------------------------------- 608.4/608.4 kB 10.8 MB/s eta 0:00:00
Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
Using cached html5lib-1.1-py2.py3-none-an


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Cargar documentos desde la carpeta
loader = DirectoryLoader("../data", show_progress=True)
documentos = loader.load()

# Dividir en fragmentos
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
chunks = splitter.split_documents(documentos)

print(f"Cantidad de chunks: {len(chunks)}")
print("Primer chunk:", chunks[0].page_content)

100%|██████████| 2/2 [00:04<00:00,  2.27s/it]

Cantidad de chunks: 46
Primer chunk: Diario de una pasion: En un hogar de retiro un hombre le lee a una mujer, que sufre de Alzheimer, la historia de dos jóvenes de distintas clases sociales que se enamoraron durante la convulsionada


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

###creacion del modelo de embedding a usar
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

### creacion del vector store a utilizar
vector_store = FAISS.from_documents(
    documents=chunks,         
    embedding=emb
)
print("Documentos cargados en FAISS")


C:\Users\arasm\AppData\Local\Temp\ipykernel_25816\429031876.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\arasm\Documentos\curso de matematica y algoritmos\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Documentos cargados en FAISS


In [ ]:
##retriever donde se extraen los 3 mas relevantes
retriever = vector_store.as_retriever(search_kwargs ={"k":3})

In [ ]:
## creacion del llm de ollama
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3", temperature=0)

In [ ]:
from langchain.prompts import PromptTemplate
##creacion del prompt
prompt = PromptTemplate(
template = """
Eres un asistente que habla acerca de las sinopsis de peliculas. Si el usuario pregunta algo que no aparece en el contexto, respondé: "Lo siento, no tengo información sobre eso" 
Contexto:
{context}

Usuario:
{input}

Responde de forma clara, concisa y en español.
"""
)

In [16]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

### creacion del contexto desde el retriever
def build_context(inputs):
    docs = retriever.get_relevant_documents(inputs["input"])
    return "\n\n".join(d.page_content for d in docs)

##pipeline para armar el RAG
final_chain = (
    RunnablePassthrough.assign(context=build_context) 
    | prompt
    | llm
    | StrOutputParser()
)



In [ ]:
from pprint import pprint #para poder imprimir el diccionario clave: valor /n clave:valor....
if __name__ == "__main__":
    questions = ["Cual es la sinopsis de superman?",
                 "Cual es la sinopsis de dr strange?",
                 "Cual es la sinopsis de un pedacito de cielo", 
                 "Cual es la sinopsis de IronMan?",
                 "Cual es la sinopsis de titanic?"]
    responses = []
    for question in questions: ##generacion de las respuestas
        resp = final_chain.invoke({"input": question})
        responses.append(resp)

    print("------ Preguntas y sus respuestas ------")
    dic = dict(zip(questions, responses))
    pprint(dic, width=150, compact=True)


------ Preguntas y sus respuestas ------
{'Cual es la sinopsis de IronMan?': 'La sinopsis de Iron Man es: Un multimillonario fabricante de armas llamado Tony Stark es secuestrado por '
                                    'terroristas que lo obligan a construir un arma devastadora. Sin embargo, después de escapar, Tony se convierte '
                                    'en Iron Man, un héroe dedicado a combatir el mal con su armadura de última generación.',
 'Cual es la sinopsis de dr strange?': 'La sinopsis de Dr. Strange es: "Dr. Stephen Strange, un famoso neurocirujano, cambia su vida después de un '
                                       'terrible accidente de coche que le arrebata el uso de sus manos. Cuando la medicina tradicional le falla, se '
                                       'enfrenta a las letales fuerzas del mal controladas por el Dr. Thaddeus Sivana."',
 'Cual es la sinopsis de superman?': 'La sinopsis de Superman es: El multimillonario tecnológico Lex Luthor aprovecha